In [1]:
!pip install torch torchvision torchaudio
!pip install scikit-learn



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.metrics import classification_report


In [3]:
class PneumoniaFilenameDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.samples = []
        self.transform = transform
        valid_ext = (".jpg", ".jpeg", ".png")

        for label_dir in ["normal", "pneumonia"]:
            folder = os.path.join(root_dir, label_dir)

            for fname in os.listdir(folder):
                if not fname.lower().endswith(valid_ext):
                    continue

                path = os.path.join(folder, fname)

                if label_dir == "normal":
                    label = 0
                else:
                    name = fname.lower()
                    if "bacteria" in name:
                        label = 1
                    elif "virus" in name:
                        label = 2
                    else:
                        continue

                self.samples.append((path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        image = Image.open(path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label


In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [5]:
train_data = PneumoniaFilenameDataset("chest_xray/train", transform)
val_data   = PneumoniaFilenameDataset("chest_xray/val", transform)
test_data  = PneumoniaFilenameDataset("chest_xray/test", transform)


In [6]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_data, batch_size=32, shuffle=False)


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [8]:


model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 3)
model = model.to(device)


c:\Users\magdu\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\magdu\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
# count samples per class
counts = torch.tensor([
    sum(1 for _, y in train_data.samples if y == 0),
    sum(1 for _, y in train_data.samples if y == 1),
    sum(1 for _, y in train_data.samples if y == 2)
], dtype=torch.float)

weights = 1.0 / counts
weights = weights / weights.sum()

criterion = nn.CrossEntropyLoss(weight=weights.to(device))


In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [11]:
for epoch in range(10):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} | Loss: {total_loss/len(train_loader):.4f}")


Epoch 1 | Loss: 0.4662
Epoch 2 | Loss: 0.2597
Epoch 3 | Loss: 0.1263
Epoch 4 | Loss: 0.0685
Epoch 5 | Loss: 0.0532
Epoch 6 | Loss: 0.0337
Epoch 7 | Loss: 0.0412
Epoch 8 | Loss: 0.0543
Epoch 9 | Loss: 0.0403
Epoch 10 | Loss: 0.0196


In [12]:
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()

        y_pred.extend(preds)
        y_true.extend(labels.numpy())

print(classification_report(
    y_true,
    y_pred,
    target_names=[
        "Normal",
        "Pneumonia (Bacterial)",
        "Pneumonia (Viral)"
    ]
))


                       precision    recall  f1-score   support

               Normal       0.98      0.53      0.69       234
Pneumonia (Bacterial)       0.81      0.95      0.88       242
    Pneumonia (Viral)       0.56      0.80      0.66       148

             accuracy                           0.76       624
            macro avg       0.78      0.76      0.74       624
         weighted avg       0.82      0.76      0.76       624



In [13]:
torch.save(model.state_dict(), "pneumonia_resnet18_3class.pt")


In [16]:
!pip install seaborn



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np


In [18]:
cm = confusion_matrix(y_true, y_pred)

labels = [
    "Normal",
    "Pneumonia (Bacterial)",
    "Pneumonia (Viral)"
]

plt.figure(figsize=(6,5))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=labels,
    yticklabels=labels
)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.tight_layout()
plt.savefig("confusion_matrix.png")
plt.close()
